# Exercise 06
# TensorFlow and Keras

### Presentado por

# Nocolás Lozada cod. 201727313
# Camilo Yate cod. 201630371

---

In [1]:
import numpy as np
import pylab as pl
from sklearn.datasets.samples_generator import make_moons

%matplotlib inline

# Functions for plotting 2D data and decision regions

def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")

def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 30)
    y_vals = np.linspace(min_y, max_y, 30)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 30, cmap = pl.cm.coolwarm, vmin= 0, vmax=1)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")

### 2. Improving the Keras text classifier

Your goal is to improve the performance of the text classifier in the Keras handout. This is are the things that you need to try:

* Different activation functions for the hidden layer (https://keras.io/activations/)
* Different optimizers (https://keras.io/optimizers/)
* Add dropout between the hidden layer and the output layer (https://keras.io/layers/core/#dropout)
* Different initializers for the dense layers (https://keras.io/initializers/)

Try different combinations and report your findings at the end. Which configuration got the best accuracy in test?


In [2]:
from keras.datasets import reuters
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
import numpy as np
import pylab as pl
from sklearn.datasets.samples_generator import make_moons
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

max_words = 1000

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Using TensorFlow backend.


Loading data...
8982 train sequences
2246 test sequences
46 classes


In [3]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
num_words = max(word_index.values()) + 1
words = ['']*num_words
for word in word_index:
    words[word_index[word]] = word
print([words[i-2] for i in x_train[101][1:]])

['', 'dlr', 'and', 'cts', '', '80', 'average', 'companies', 'in', 'income', 'of', 'make', '', '', 'said', '', '', 'a', 'of', 'make', '52', '', 'said', '', 'of', '1987', '', '2', 'of', 'sold', 'general', 'states', 'to', '', 'field', 'securities', 'was', 'agricultural', '', '3', 'it', 'a', '1988', 'said', 'as', 'april', '50', 'term', 'to', 'earlier', '3', 'it', 'but', 'was', 'with', '', 'said', '', 'previously', 'be', 'sell', 'cts', 'previously', 'be', '', 'more', 'earlier', 'of', 'which', 'and', 'said', 'commerce', 'of', '1987', 'was', '', 'august', '3', 'it', 'export', 'april', 'report', 'vice', 'to', 'beef', '3', 'it', '', 'and', '000', 'for']


In [4]:
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [6]:
model = Sequential()
model.add(Dense(256, input_shape=(max_words,)))
model.add(Activation('sigmoid'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary(70)

______________________________________________________________________
Layer (type)                   Output Shape                Param #    
dense_1 (Dense)                (None, 256)                 256256     
______________________________________________________________________
activation_1 (Activation)      (None, 256)                 0          
______________________________________________________________________
dense_2 (Dense)                (None, 46)                  11822      
______________________________________________________________________
activation_2 (Activation)      (None, 46)                  0          
Total params: 268,078
Trainable params: 268,078
Non-trainable params: 0
______________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [8]:
batch_size = 32
epochs = 5
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 1s 172us/step - loss: 2.4542 - acc: 0.3630 - val_loss: 2.3069 - val_acc: 0.4260
Epoch 2/5
8083/8083 [==============================] - 1s 148us/step - loss: 2.1809 - acc: 0.4397 - val_loss: 2.1788 - val_acc: 0.4750
Epoch 3/5
8083/8083 [==============================] - 1s 145us/step - loss: 2.0693 - acc: 0.4876 - val_loss: 2.0863 - val_acc: 0.4905
Epoch 4/5
8083/8083 [==============================] - 1s 146us/step - loss: 1.9903 - acc: 0.5032 - val_loss: 2.0180 - val_acc: 0.5061
Epoch 5/5
8083/8083 [==============================] - 1s 144us/step - loss: 1.9276 - acc: 0.5163 - val_loss: 1.9608 - val_acc: 0.5150


In [9]:
score = model.evaluate(x_test, y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

2246/2246 [==============================] - 0s 58us/step
Test score: 1.92646697049
Test accuracy: 0.52582368658


# diferentes tipos de activación capa oculta



* softmax
* relu
* tanh
* linear
* elu



### oftmax

In [10]:
model_softamax = Sequential()
model_softamax.add(Dense(256, input_shape=(max_words,)))
model_softamax.add(Activation('softmax'))
model_softamax.add(Dense(num_classes))
model_softamax.add(Activation('softmax'))
model_softamax.summary(70)

______________________________________________________________________
Layer (type)                   Output Shape                Param #    
dense_3 (Dense)                (None, 256)                 256256     
______________________________________________________________________
activation_3 (Activation)      (None, 256)                 0          
______________________________________________________________________
dense_4 (Dense)                (None, 46)                  11822      
______________________________________________________________________
activation_4 (Activation)      (None, 46)                  0          
Total params: 268,078
Trainable params: 268,078
Non-trainable params: 0
______________________________________________________________________


In [11]:
model_softamax.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [12]:
batch_size = 32
epochs = 5
history = model_softamax.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 219us/step - loss: 3.6315 - acc: 0.3475 - val_loss: 3.4569 - val_acc: 0.3315
Epoch 2/5
8083/8083 [==============================] - 2s 192us/step - loss: 3.2789 - acc: 0.3540 - val_loss: 3.1575 - val_acc: 0.3315
Epoch 3/5
8083/8083 [==============================] - 2s 195us/step - loss: 3.0113 - acc: 0.3540 - val_loss: 2.9468 - val_acc: 0.3315
Epoch 4/5
8083/8083 [==============================] - 2s 197us/step - loss: 2.8327 - acc: 0.3540 - val_loss: 2.8144 - val_acc: 0.3315
Epoch 5/5
8083/8083 [==============================] - 2s 196us/step - loss: 2.7230 - acc: 0.3540 - val_loss: 2.7342 - val_acc: 0.3315


In [13]:
score_softmax = model_softamax.evaluate(x_test, y_test)
print('Test score:', score_softmax[0])
print('Test accuracy:', score_softmax[1])

2246/2246 [==============================] - 0s 95us/step
Test score: 2.6777455794
Test accuracy: 0.361976847782


### relu

In [14]:
model_relu = Sequential()
model_relu.add(Dense(256, input_shape=(max_words,)))
model_relu.add(Activation('relu'))
model_relu.add(Dense(num_classes))
model_relu.add(Activation('softmax'))
model_relu.summary(70)

______________________________________________________________________
Layer (type)                   Output Shape                Param #    
dense_5 (Dense)                (None, 256)                 256256     
______________________________________________________________________
activation_5 (Activation)      (None, 256)                 0          
______________________________________________________________________
dense_6 (Dense)                (None, 46)                  11822      
______________________________________________________________________
activation_6 (Activation)      (None, 46)                  0          
Total params: 268,078
Trainable params: 268,078
Non-trainable params: 0
______________________________________________________________________


In [15]:
model_relu.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [16]:
batch_size = 32
epochs = 5
history = model_relu.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 1s 176us/step - loss: 2.4796 - acc: 0.4595 - val_loss: 1.9839 - val_acc: 0.5139
Epoch 2/5
8083/8083 [==============================] - 1s 148us/step - loss: 1.7881 - acc: 0.5581 - val_loss: 1.7584 - val_acc: 0.5895
Epoch 3/5
8083/8083 [==============================] - 1s 146us/step - loss: 1.6135 - acc: 0.6138 - val_loss: 1.6383 - val_acc: 0.6363
Epoch 4/5
8083/8083 [==============================] - 1s 146us/step - loss: 1.4953 - acc: 0.6567 - val_loss: 1.5516 - val_acc: 0.6652
Epoch 5/5
8083/8083 [==============================] - 1s 146us/step - loss: 1.4022 - acc: 0.6830 - val_loss: 1.4841 - val_acc: 0.6830


In [17]:
score_relu = model_relu.evaluate(x_test, y_test)
print('Test score:', score_relu[0])
print('Test accuracy:', score_relu[1])

2246/2246 [==============================] - 0s 61us/step
Test score: 1.46068778191
Test accuracy: 0.674977738228


### tanh

In [18]:
model_tanh = Sequential()
model_tanh.add(Dense(256, input_shape=(max_words,)))
model_tanh.add(Activation('tanh'))
model_tanh.add(Dense(num_classes))
model_tanh.add(Activation('softmax'))
#model_elu.summary(70)

In [19]:
model_tanh.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [20]:
batch_size = 32
epochs = 5
history = model_tanh.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 1s 184us/step - loss: 2.2105 - acc: 0.4865 - val_loss: 1.8361 - val_acc: 0.5451
Epoch 2/5
8083/8083 [==============================] - 1s 161us/step - loss: 1.6702 - acc: 0.5945 - val_loss: 1.6420 - val_acc: 0.6207
Epoch 3/5
8083/8083 [==============================] - 1s 151us/step - loss: 1.5056 - acc: 0.6573 - val_loss: 1.5246 - val_acc: 0.6674
Epoch 4/5
8083/8083 [==============================] - 1s 165us/step - loss: 1.3938 - acc: 0.6900 - val_loss: 1.4466 - val_acc: 0.6874
Epoch 5/5
8083/8083 [==============================] - 1s 153us/step - loss: 1.3087 - acc: 0.7120 - val_loss: 1.3864 - val_acc: 0.6986


In [21]:
score_tanh = model_tanh.evaluate(x_test, y_test)
print('Test score:', score_tanh[0])
print('Test accuracy:', score_tanh[1])

2246/2246 [==============================] - 0s 64us/step
Test score: 1.37898335983
Test accuracy: 0.690560997355


### linear

In [22]:
model_linear = Sequential()
model_linear.add(Dense(256, input_shape=(max_words,)))
model_linear.add(Activation('linear'))
model_linear.add(Dense(num_classes))
model_linear.add(Activation('softmax'))
#model_elu.summary(70)

In [23]:
model_linear.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [24]:
batch_size = 32
epochs = 5
history = model_linear.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 1s 181us/step - loss: 2.1693 - acc: 0.4951 - val_loss: 1.8140 - val_acc: 0.5617
Epoch 2/5
8083/8083 [==============================] - 1s 155us/step - loss: 1.6270 - acc: 0.6188 - val_loss: 1.6128 - val_acc: 0.6407
Epoch 3/5
8083/8083 [==============================] - 1s 152us/step - loss: 1.4580 - acc: 0.6686 - val_loss: 1.5002 - val_acc: 0.6674
Epoch 4/5
8083/8083 [==============================] - 1s 159us/step - loss: 1.3458 - acc: 0.6960 - val_loss: 1.4216 - val_acc: 0.6863
Epoch 5/5
8083/8083 [==============================] - 1s 164us/step - loss: 1.2620 - acc: 0.7209 - val_loss: 1.3699 - val_acc: 0.7052


In [25]:
score_linear = model_linear.evaluate(x_test, y_test)
print('Test score:', score_linear[0])
print('Test accuracy:', score_linear[1])

2246/2246 [==============================] - 0s 67us/step
Test score: 1.34297478443
Test accuracy: 0.707925200383


### elu

In [26]:
model_elu = Sequential()
model_elu.add(Dense(256, input_shape=(max_words,)))
model_elu.add(Activation('elu'))
model_elu.add(Dense(num_classes))
model_elu.add(Activation('softmax'))
#model_elu.summary(70)

In [27]:
model_elu.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [28]:
batch_size = 32
epochs = 5
history = model_elu.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 198us/step - loss: 2.2325 - acc: 0.5034 - val_loss: 1.8474 - val_acc: 0.5439
Epoch 2/5
8083/8083 [==============================] - 1s 163us/step - loss: 1.6604 - acc: 0.6039 - val_loss: 1.6544 - val_acc: 0.6218
Epoch 3/5
8083/8083 [==============================] - 1s 162us/step - loss: 1.4976 - acc: 0.6563 - val_loss: 1.5438 - val_acc: 0.6674
Epoch 4/5
8083/8083 [==============================] - 1s 161us/step - loss: 1.3875 - acc: 0.6842 - val_loss: 1.4662 - val_acc: 0.6841
Epoch 5/5
8083/8083 [==============================] - 1s 163us/step - loss: 1.3029 - acc: 0.7073 - val_loss: 1.4070 - val_acc: 0.6952


In [29]:
score_elu = model_elu.evaluate(x_test, y_test)
print('Test score:', score_elu[0])
print('Test accuracy:', score_elu[1])

2246/2246 [==============================] - 0s 68us/step
Test score: 1.37529943505
Test accuracy: 0.694122885156


## Resumen

A contniuacuón se muestra el accuracy de los distintoss modelos utilizando distintas activaciones para la capa oculta

In [30]:
print('Test accuracy elu:', score_elu[1])
print('Test accuracy relu:', score_relu[1])
print('Test accuracy linear:', score_linear[1])
print('Test accuracy tanh:', score_tanh[1])
print('Test accuracysoftmax :', score_softmax[1])
print('Test accuracy sigmoid:', score[1])

Test accuracy elu: 0.694122885156
Test accuracy relu: 0.674977738228
Test accuracy linear: 0.707925200383
Test accuracy tanh: 0.690560997355
Test accuracysoftmax : 0.361976847782
Test accuracy sigmoid: 0.52582368658


# Probar con distintos optimizadores
* RMSprop
* Adagrad
* Adadelta
* Adam

A partir de aquí tomamos la activación linear en la capa oculta

### RMSprop

In [31]:
model_2_RMSprop = Sequential()
model_2_RMSprop.add(Dense(256, input_shape=(max_words,)))
model_2_RMSprop.add(Activation('linear'))
model_2_RMSprop.add(Dense(num_classes))
model_2_RMSprop.add(Activation('softmax'))


In [32]:
from keras.optimizers import RMSprop
model_2_RMSprop.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [33]:
batch_size = 32
epochs = 5
history = model_2_RMSprop.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 282us/step - loss: 1.2143 - acc: 0.7312 - val_loss: 1.0160 - val_acc: 0.7820
Epoch 2/5
8083/8083 [==============================] - 2s 264us/step - loss: 0.6651 - acc: 0.8492 - val_loss: 0.9190 - val_acc: 0.7864
Epoch 3/5
8083/8083 [==============================] - 2s 280us/step - loss: 0.4798 - acc: 0.8914 - val_loss: 0.9563 - val_acc: 0.7987
Epoch 4/5
8083/8083 [==============================] - 3s 357us/step - loss: 0.3780 - acc: 0.9101 - val_loss: 1.0506 - val_acc: 0.7998
Epoch 5/5
8083/8083 [==============================] - 3s 322us/step - loss: 0.3232 - acc: 0.9240 - val_loss: 1.0710 - val_acc: 0.7853


In [34]:
score_2_RMSprop = model_2_RMSprop.evaluate(x_test, y_test)
print('Test score:', score_2_RMSprop[0])
print('Test accuracy:', score_2_RMSprop[1])

2246/2246 [==============================] - 0s 82us/step
Test score: 1.01439451492
Test accuracy: 0.788512911843


### Adagrad

In [35]:
model_2_Adagrad = Sequential()
model_2_Adagrad.add(Dense(256, input_shape=(max_words,)))
model_2_Adagrad.add(Activation('linear'))
model_2_Adagrad.add(Dense(num_classes))
model_2_Adagrad.add(Activation('softmax'))


In [36]:
from keras.optimizers import Adagrad
model_2_Adagrad.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

In [37]:
batch_size = 32
epochs = 5
history = model_2_Adagrad.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 304us/step - loss: 1.0993 - acc: 0.7575 - val_loss: 0.9505 - val_acc: 0.7864
Epoch 2/5
8083/8083 [==============================] - 2s 246us/step - loss: 0.6001 - acc: 0.8666 - val_loss: 0.9134 - val_acc: 0.7964
Epoch 3/5
8083/8083 [==============================] - 2s 263us/step - loss: 0.4721 - acc: 0.8952 - val_loss: 0.8786 - val_acc: 0.8098
Epoch 4/5
8083/8083 [==============================] - 2s 192us/step - loss: 0.3994 - acc: 0.9123 - val_loss: 0.8664 - val_acc: 0.8053
Epoch 5/5
8083/8083 [==============================] - 2s 227us/step - loss: 0.3490 - acc: 0.9230 - val_loss: 0.8879 - val_acc: 0.8131


In [38]:
score_2_Adagrad = model_2_Adagrad.evaluate(x_test, y_test)
print('Test score:', score_2_Adagrad[0])
print('Test accuracy:', score_2_Adagrad[1])

2246/2246 [==============================] - 0s 94us/step
Test score: 0.871606975396
Test accuracy: 0.796081923419


### Adadelta

In [39]:
model_2_Adadelta = Sequential()
model_2_Adadelta.add(Dense(256, input_shape=(max_words,)))
model_2_Adadelta.add(Activation('linear'))
model_2_Adadelta.add(Dense(num_classes))
model_2_Adadelta.add(Activation('softmax'))


In [40]:
from keras.optimizers import Adadelta
model_2_Adadelta.compile(loss='categorical_crossentropy',
              optimizer=Adadelta(),
              metrics=['accuracy'])

In [41]:
batch_size = 32
epochs = 5
history = model_2_Adadelta.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 3s 385us/step - loss: 1.2710 - acc: 0.7194 - val_loss: 1.0751 - val_acc: 0.7597
Epoch 2/5
8083/8083 [==============================] - 3s 358us/step - loss: 0.7258 - acc: 0.8331 - val_loss: 0.9826 - val_acc: 0.7842
Epoch 3/5
8083/8083 [==============================] - 3s 373us/step - loss: 0.5395 - acc: 0.8781 - val_loss: 0.9522 - val_acc: 0.7875
Epoch 4/5
8083/8083 [==============================] - 3s 330us/step - loss: 0.4355 - acc: 0.9003 - val_loss: 1.0073 - val_acc: 0.7820
Epoch 5/5
8083/8083 [==============================] - 3s 391us/step - loss: 0.3649 - acc: 0.9130 - val_loss: 0.9485 - val_acc: 0.7898


In [42]:
score_2_Adadelta = model_2_Adadelta.evaluate(x_test, y_test)
print('Test score:', score_2_Adadelta[0])
print('Test accuracy:', score_2_Adadelta[1])

2246/2246 [==============================] - 0s 80us/step
Test score: 0.948198899136
Test accuracy: 0.786731967996


### Adam

In [43]:
model_2_Adam = Sequential()
model_2_Adam.add(Dense(256, input_shape=(max_words,)))
model_2_Adam.add(Activation('linear'))
model_2_Adam.add(Dense(num_classes))
model_2_Adam.add(Activation('softmax'))


In [44]:
from keras.optimizers import Adam
model_2_Adam.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [45]:
batch_size = 32
epochs = 5
history = model_2_Adam.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 4s 529us/step - loss: 1.2453 - acc: 0.7281 - val_loss: 1.0016 - val_acc: 0.7775
Epoch 2/5
8083/8083 [==============================] - 3s 350us/step - loss: 0.6299 - acc: 0.8578 - val_loss: 0.8984 - val_acc: 0.8031
Epoch 3/5
8083/8083 [==============================] - 3s 338us/step - loss: 0.4461 - acc: 0.8961 - val_loss: 0.9393 - val_acc: 0.7942
Epoch 4/5
8083/8083 [==============================] - 3s 335us/step - loss: 0.3484 - acc: 0.9157 - val_loss: 0.9572 - val_acc: 0.8042
Epoch 5/5
8083/8083 [==============================] - 2s 285us/step - loss: 0.2901 - acc: 0.9287 - val_loss: 1.0062 - val_acc: 0.7942


In [46]:
score_2_Adam = model_2_Adam.evaluate(x_test, y_test)
print('Test score:', score_2_Adam[0])
print('Test accuracy:', score_2_Adam[1])

2246/2246 [==============================] - 0s 61us/step
Test score: 0.974930532044
Test accuracy: 0.78272484422


## resumen

A contninuación se muestran los accuracy de los modelo con los distintos tipos de optimizadores utilizados

In [47]:
print('Test accuracy score_2_RMSprop:', score_2_RMSprop[1])
print('Test accuracy Adagrad:', score_2_Adagrad[1])
print('Test accuracy Adadelta:', score_2_Adadelta[1])
print('Test accuracy Adam:', score_2_Adam[1])


Test accuracy score_2_RMSprop: 0.788512911843
Test accuracy Adagrad: 0.796081923419
Test accuracy Adadelta: 0.786731967996
Test accuracy Adam: 0.78272484422


### DROPOUT

A contniuación se incluye un dropout, con una tasa de 0.5, conservando la activación linear en la capa oculta y el optimizador Adagrad que hasta ahora arroja el mejor resultado.



In [48]:
model_3_dropout = Sequential()
model_3_dropout.add(Dense(256, input_shape=(max_words,)))
model_3_dropout.add(Activation('linear'))
model_3_dropout.add(Dropout(0.5))
model_3_dropout.add(Dense(num_classes))
model_3_dropout.add(Activation('softmax'))


In [49]:

model_3_dropout.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

In [50]:
batch_size = 32
epochs = 5
history = model_3_dropout.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 245us/step - loss: 1.2185 - acc: 0.7328 - val_loss: 1.0295 - val_acc: 0.7775
Epoch 2/5
8083/8083 [==============================] - 2s 209us/step - loss: 0.7600 - acc: 0.8309 - val_loss: 0.9446 - val_acc: 0.7887
Epoch 3/5
8083/8083 [==============================] - 2s 204us/step - loss: 0.6310 - acc: 0.8567 - val_loss: 0.9170 - val_acc: 0.8042
Epoch 4/5
8083/8083 [==============================] - 2s 206us/step - loss: 0.5586 - acc: 0.8734 - val_loss: 0.9026 - val_acc: 0.8020
Epoch 5/5
8083/8083 [==============================] - 2s 257us/step - loss: 0.4931 - acc: 0.8866 - val_loss: 0.8960 - val_acc: 0.8087


In [51]:
score_3_dropout = model_3_dropout.evaluate(x_test, y_test)
print('Test score:', score_3_dropout[0])
print('Test accuracy:', score_3_dropout[1])

2246/2246 [==============================] - 0s 63us/step
Test score: 0.8683658523
Test accuracy: 0.79697239537


# inicializador capa denza

A continuación se muestran distintios inicializadores , con activación linear en la capa oculta y optimizador Adagrad

* RandomUniform
* RandomNormal
* TruncatedNormal
* VarianceScaling


### score_4_RandomUniform

In [52]:
model_4_RandomUniform = Sequential()
model_4_RandomUniform.add(Dense(256, input_shape=(max_words,),kernel_initializer = 'RandomUniform'))
model_4_RandomUniform.add(Activation('linear'))
model_4_RandomUniform.add(Dropout(0.5))
model_4_RandomUniform.add(Dense(num_classes))
model_4_RandomUniform.add(Activation('softmax'))


In [53]:

model_4_RandomUniform.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

In [54]:
batch_size = 32
epochs = 5
history = model_4_RandomUniform.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 247us/step - loss: 1.1959 - acc: 0.7367 - val_loss: 1.0327 - val_acc: 0.7786
Epoch 2/5
8083/8083 [==============================] - 2s 230us/step - loss: 0.7416 - acc: 0.8305 - val_loss: 0.9486 - val_acc: 0.7942
Epoch 3/5
8083/8083 [==============================] - 2s 206us/step - loss: 0.6124 - acc: 0.8617 - val_loss: 0.9149 - val_acc: 0.7976
Epoch 4/5
8083/8083 [==============================] - 2s 243us/step - loss: 0.5392 - acc: 0.8790 - val_loss: 0.9018 - val_acc: 0.7998
Epoch 5/5
8083/8083 [==============================] - 2s 275us/step - loss: 0.4844 - acc: 0.8883 - val_loss: 0.8851 - val_acc: 0.8042


In [55]:
score_4_RandomUniform = model_4_RandomUniform.evaluate(x_test, y_test)
print('Test score:', score_4_RandomUniform[0])
print('Test accuracy:', score_4_RandomUniform[1])

2246/2246 [==============================] - 0s 107us/step
Test score: 0.858450887464
Test accuracy: 0.798308103321


### RandomNormal

In [56]:
model_4_RandomNormal = Sequential()
model_4_RandomNormal.add(Dense(256, input_shape=(max_words,),kernel_initializer = 'RandomNormal'))
model_4_RandomNormal.add(Activation('linear'))
model_4_RandomNormal.add(Dropout(0.5))
model_4_RandomNormal.add(Dense(num_classes))
model_4_RandomNormal.add(Activation('softmax'))


In [57]:

model_4_RandomNormal.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

In [58]:
batch_size = 32
epochs = 5
history = model_4_RandomNormal.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 3s 367us/step - loss: 1.2478 - acc: 0.7251 - val_loss: 1.0512 - val_acc: 0.7742
Epoch 2/5
8083/8083 [==============================] - 3s 313us/step - loss: 0.7822 - acc: 0.8218 - val_loss: 0.9652 - val_acc: 0.7909
Epoch 3/5
8083/8083 [==============================] - 3s 321us/step - loss: 0.6553 - acc: 0.8484 - val_loss: 0.9383 - val_acc: 0.7931
Epoch 4/5
8083/8083 [==============================] - 3s 317us/step - loss: 0.5794 - acc: 0.8666 - val_loss: 0.9142 - val_acc: 0.8009
Epoch 5/5
8083/8083 [==============================] - 2s 266us/step - loss: 0.5210 - acc: 0.8788 - val_loss: 0.9108 - val_acc: 0.7964


In [59]:
score_4_RandomNormal = model_4_RandomNormal.evaluate(x_test, y_test)
print('Test score:', score_4_RandomNormal[0])
print('Test accuracy:', score_4_RandomNormal[1])

2246/2246 [==============================] - 0s 89us/step
Test score: 0.86388503157
Test accuracy: 0.79964381122


### TruncatedNormal

In [60]:
model_4_TruncatedNormal = Sequential()
model_4_TruncatedNormal.add(Dense(256, input_shape=(max_words,),kernel_initializer = 'TruncatedNormal'))
model_4_TruncatedNormal.add(Activation('linear'))
model_4_TruncatedNormal.add(Dropout(0.5))
model_4_TruncatedNormal.add(Dense(num_classes))
model_4_TruncatedNormal.add(Activation('softmax'))


In [61]:

model_4_TruncatedNormal.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

In [62]:
batch_size = 32
epochs = 5
history = model_4_TruncatedNormal.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 283us/step - loss: 1.2390 - acc: 0.7286 - val_loss: 1.0385 - val_acc: 0.7731
Epoch 2/5
8083/8083 [==============================] - 2s 245us/step - loss: 0.7668 - acc: 0.8277 - val_loss: 0.9617 - val_acc: 0.7887
Epoch 3/5
8083/8083 [==============================] - 2s 220us/step - loss: 0.6494 - acc: 0.8524 - val_loss: 0.9253 - val_acc: 0.7898
Epoch 4/5
8083/8083 [==============================] - 2s 217us/step - loss: 0.5663 - acc: 0.8731 - val_loss: 0.9082 - val_acc: 0.7942
Epoch 5/5
8083/8083 [==============================] - 2s 251us/step - loss: 0.5080 - acc: 0.8823 - val_loss: 0.8981 - val_acc: 0.8031


In [63]:
score_4_TruncatedNormal = model_4_TruncatedNormal.evaluate(x_test, y_test)
print('Test score:', score_4_TruncatedNormal[0])
print('Test accuracy:', score_4_TruncatedNormal[1])

2246/2246 [==============================] - 0s 68us/step
Test score: 0.864811734864
Test accuracy: 0.793410507569


### VarianceScaling

In [64]:
model_4_VarianceScaling = Sequential()
model_4_VarianceScaling.add(Dense(256, input_shape=(max_words,),kernel_initializer = 'VarianceScaling'))
model_4_VarianceScaling.add(Activation('linear'))
model_4_VarianceScaling.add(Dropout(0.5))
model_4_VarianceScaling.add(Dense(num_classes))
model_4_VarianceScaling.add(Activation('softmax'))


In [65]:

model_4_VarianceScaling.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

In [66]:
batch_size = 32
epochs = 5
history = model_4_VarianceScaling.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 2s 253us/step - loss: 1.2047 - acc: 0.7348 - val_loss: 1.0039 - val_acc: 0.7753
Epoch 2/5
8083/8083 [==============================] - 2s 206us/step - loss: 0.7590 - acc: 0.8316 - val_loss: 0.9515 - val_acc: 0.7909
Epoch 3/5
8083/8083 [==============================] - 2s 209us/step - loss: 0.6203 - acc: 0.8622 - val_loss: 0.9146 - val_acc: 0.7998
Epoch 4/5
8083/8083 [==============================] - 2s 211us/step - loss: 0.5447 - acc: 0.8754 - val_loss: 0.8958 - val_acc: 0.8009
Epoch 5/5
8083/8083 [==============================] - 2s 209us/step - loss: 0.4997 - acc: 0.8849 - val_loss: 0.8862 - val_acc: 0.8031


In [67]:
score_4_VarianceScaling = model_4_VarianceScaling.evaluate(x_test, y_test)
print('Test score:', score_4_VarianceScaling[0])
print('Test accuracy:', score_4_VarianceScaling[1])

2246/2246 [==============================] - 0s 62us/step
Test score: 0.859848294848
Test accuracy: 0.796527159394


# Resumen

A continuación el accuracy de los modelos con diferentes inicializadores

In [68]:
print('Test accuracy RandomNormal:', score_4_RandomNormal[1])
print('Test accuracy RandomUniform:', score_4_RandomUniform[1])
print('Test accuracy TruncatedNormal:', score_4_TruncatedNormal[1])
print('Test accuracy VarianceScaling:', score_4_VarianceScaling[1])


Test accuracy RandomNormal: 0.79964381122
Test accuracy RandomUniform: 0.798308103321
Test accuracy TruncatedNormal: 0.793410507569
Test accuracy VarianceScaling: 0.796527159394


# Conclusiones

El primer modelo proporcionado arrojaba un accuracy de 0.52 en test. luego de probar distintas formas de activación de las neuronas, distintos optimizadores e incializadores, logramos un accuracy de 0.79 con activación linear en la capa oculta, optimizador Adagrat, Droopout de 0.5 y intializador dRandomNormal.